In [28]:
import time
import praw
from datetime import datetime
import threading
import pandas as pd

In [2]:
reddit = praw.Reddit(client_id='P26uUpjfnflDVg', client_secret="L9oHqikaIVKSm-aXv-a7E-x8Yi8",
                     password='password', user_agent='script by /u/CandlesTaken',
                     username='username')

In [3]:
class Post:
    commentforrest = reddit.submission('3hahrw').comments
    comment = []
    upvote_count = 0
    def __init__(self, subreddit, title, created_time,Pid):
        self.subreddit = subreddit
        self.title = title
        self.created_time = created_time
        self.Pid = Pid

In [4]:
class Comment:
    def __init__(self, body,Cid):
        self.Cid = Cid
        self.body = body

In [12]:
def submissionstream(subreddits,submissions):#stream#
    string = str("")
    for i in range(0,len(subreddits)):
        if (i != 0):
            string += "+"+str(subreddits[i].title)
        else:
            string += str(subreddits[i].title)
    for submission in reddit.subreddit(string).stream.submissions():
        print(str(submission.subreddit.title) + "; " + submission.title+"; "+str(datetime.utcfromtimestamp(submission.created_utc))+";"+str(submission.score))
        dt = datetime.utcfromtimestamp(submission.created_utc)
        post = Post(submission.subreddit.title, submission.title, dt, submission.id)
        submissions.append(post)
    return submissions

In [25]:
def submissionsupdate(subreddits,submissions):#called at the end of the day#
    for submission in submissions:
        post = reddit.submission(id=submission.Pid)
        submission.upvote_count = post.score
        submission.commentforrest = post.comments
        for comment in submission.commentforrest:
            submission.comment.append(Comment(comment.body, comment.id))
    print("update_done!")
    return submissions

In [14]:
def hot(subreddits,hots):
    for sub in subreddits:
        hotposts = sub.hot(limit = 50)
        for submission in hotposts:
            dt = datetime.utcfromtimestamp(submission.created_utc)
            post = Post(submission.subreddit.title, submission.title, dt, submission.id)
            post.upvote_count = submission.score
            hots.append(post)
        print(sub.title+" done!")
    return hots, datetime.utcnow()

In [48]:
def csvwriter(submissions,hots):
    stream_dic = {"subreddit":[],
                  "Pid":[],
                  "title":[],
                  "upvote":[],
                  "date":[]}
    comment_dic = {"Cid":[],
                   "Pid":[],
                   "body":[]}
    hot_dic = {"subreddit":[],
               "Pid":[],
               "title":[],
               "upvote":[],
               "date":[]}
    for post in submissions:
        stream_dic["subreddit"].append(post.subreddit)
        stream_dic["Pid"].append(post.Pid)
        stream_dic["title"].append(post.title)
        stream_dic["upvote"].append(post.upvote_count)
        stream_dic["date"].append(post.created_time)
        for comment in post.comment:
            comment_dic["Cid"].append(comment.Cid)
            comment_dic["Pid"].append(post.Pid)
            comment_dic["body"].append(comment.body)
    for post in hots:
        hot_dic["subreddit"].append(post.subreddit)
        hot_dic["Pid"].append(post.Pid)
        hot_dic["title"].append(post.title)
        hot_dic["upvote"].append(post.upvote_count)
        hot_dic["date"].append(post.created_time)
    dt = str(datetime.utcnow()).split(" ")[0]
    df_stream = pd.DataFrame(stream_dic)
    df_comment = pd.DataFrame(comment_dic)
    df_hot = pd.DataFrame(hot_dic)
    df_stream.to_csv(str('stream'+dt+'.csv'))
    df_comment.to_csv(str('comment'+dt+'.csv'))
    df_hot.to_csv(str('hot'+dt+'.csv'))

In [15]:
subreddits = [reddit.subreddit('CoronavirusUS'),reddit.subreddit('CoronavirusUK'),reddit.subreddit('CoronavirusNewYork'),reddit.subreddit('CoronavirusCA'),reddit.subreddit('CoronavirusMichigan')]

In [16]:
submissions = []
hots = []

In [22]:
submissionstream(subreddits,submissions)

CoronavirusUS; Watch Now : Trump: "We have to get our country open." Question: "Will you say, sir, what metrics you will use to make that decision?" Trump: "The metric's right here." [Points to head]; 2020-04-11 15:29:42;73
CoronavirusUS; Walmart CEO says we're in the 'hair color' phase of panic buying; 2020-04-11 15:31:54;47
CoronavirusNewYork; Has anyone received any calls yet?; 2020-04-11 15:33:34;4
CoronavirusNewYork; Coronavirus in New York: Latest Updates; 2020-04-11 15:38:03;5
CoronavirusUK; Force removal of Early Termination Clauses?; 2020-04-11 15:40:40;1
CoronavirusUS; A plan to defeat coronavirus finally emerges, but it’s not from the White House; 2020-04-11 15:43:39;0
CoronavirusUS; Help Relaying the Message; 2020-04-11 15:46:03;0
CoronavirusUS; When does this end? Open up colleges?; 2020-04-11 15:48:25;0
CoronavirusUK; Facing a huge moral dilemma and need help; 2020-04-11 15:48:56;14
CoronavirusUS; Indiana government count of COVID-19 deaths versus John Hopkins dataset; 20

KeyboardInterrupt: 

In [24]:
submissionsupdate(subreddits,submissions)

update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!
update_done!

NameError: name 'comments' is not defined

In [26]:
hot(subreddits,hots)

doing hots!
CoronavirusUS; Daily Q&A - Ask + Answer Questions Here; 2020-04-10 18:50:32;9
CoronavirusUS; Let's compare notes: Anyone experiencing symptoms but refused testing?; 2020-04-07 22:36:31;110
CoronavirusUS; Stimulus package; 2020-04-11 16:10:38;919
CoronavirusUS; I Called out; 2020-04-11 12:13:15;1315
CoronavirusUS; I’ve never thought that I would actually say this, but I actually somewhat enjoy the quarantine life now. Life is just so much simpler and slower now that I don’t have to always go somewhere or deal with people all the time. I genuinely have more time to focus more on myself and what makes me happy.; 2020-04-11 21:58:23;154
CoronavirusUS; Shoutout to all healthcare team members, many of whom are often unknown/underappreciated; 2020-04-11 20:13:27;199
CoronavirusUS; Prepare for the Ultimate Gaslighting; 2020-04-11 17:50:25;66
CoronavirusUS; Fake ThermoScan from china that will never exceed 37℃; 2020-04-11 20:25:59;44
CoronavirusUS; Watch Now : Trump: "We have to get

([<__main__.Post at 0x1b0cc417048>,
 datetime.datetime(2020, 4, 12, 0, 11, 9, 280167))

In [49]:
csvwriter(submissions,hots)